![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [ ]:
model_id = "ibm/granite-20b-code-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 1000,
    "temperature": 0.71,
    "top_k": 50,
    "top_p": 1,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [ ]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:


In [ ]:
prompt_input = """Analyze the following function or class and generate detailed documentation. The documentation should include:

1. A clear description of what the function or class does.
2. The purpose and data types of the input arguments.
3. The return value and its data type.
4. Any exceptions or errors that could be raised.
5. An example of how to use the function or class in practice.
6. For classes, also document each method separately, including a brief explanation of the class itself."

Input: def divide_numbers(a, b):
    return a / b

Output: def divide_numbers(a, b):
    """
    Divide two numbers and return the result.

    Args:
        a (float): The dividend (numerator).
        b (float): The divisor (denominator), must be non-zero.

    Returns:
        float: The result of dividing a by b.

    Raises:
        ZeroDivisionError: If b is zero.

    Example:
        >>> divide_numbers(10, 2)
        5.0
        >>> divide_numbers(10, 0)
        ZeroDivisionError: division by zero
    """
    return a / b


Input: function findMax(arr) {
    let max = arr[0];
    for (let i = 1; i < arr.length; i++) {
        if (arr[i] > max) {
            max = arr[i];
        }
    }
    return max;
}

Output: /**
 * Find the maximum value in an array of numbers.
 * 
 * @param {number[]} arr - An array of numbers to find the maximum value from.
 * @returns {number} The maximum value in the array.
 * 
 * @throws {Error} If the array is empty.
 * 
 * Example:
 *    findMax([1, 3, 2, 5, 4]); // returns 5
 *    findMax([]); // throws Error: Array is empty
 */
function findMax(arr) {
    if (arr.length === 0) {
        throw new Error("Array is empty");
    }
    
    let max = arr[0];
    for (let i = 1; i < arr.length; i++) {
        if (arr[i] > max) {
            max = arr[i];
        }
    }
    return max;
}


Input: class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        return self.balance

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        return self.balance

Output: class BankAccount:
    """
    A class to represent a bank account.

    Attributes:
        owner (str): The name of the account owner.
        balance (float): The account balance, initialized to 0 by default.

    Methods:
        deposit(amount): Add funds to the account and return the new balance.
        withdraw(amount): Withdraw funds from the account and return the new balance.
    """

    def __init__(self, owner, balance=0):
        """
        Initialize the BankAccount with an owner and an optional initial balance.

        Args:
            owner (str): The name of the account owner.
            balance (float, optional): The initial account balance. Defaults to 0.
        
        Example:
            >>> account = BankAccount('John', 100)
            >>> account.owner
            'John'
            >>> account.balance
            100
        """
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        """
        Deposit the specified amount into the account and return the new balance.

        Args:
            amount (float): The amount to deposit.

        Returns:
            float: The updated account balance.

        Example:
            >>> account.deposit(50)
            150
        """
        self.balance += amount
        return self.balance

    def withdraw(self, amount):
        """
        Withdraw the specified amount from the account.

        Args:
            amount (float): The amount to withdraw.

        Returns:
            float: The updated account balance.

        Raises:
            ValueError: If the amount is greater than the available balance.

        Example:
            >>> account.withdraw(50)
            100
            >>> account.withdraw(200)
            ValueError: Insufficient funds
        """
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        return self.balance


Input: public class Rectangle {
    private int width;
    private int height;

    public Rectangle(int width, int height) {
        this.width = width;
        this.height = height;
    }

    public int area() {
        return width * height;
    }

    public int perimeter() {
        return 2 * (width + height);
    }
}

Output: /**
 * A class to represent a rectangle and calculate its area and perimeter.
 * 
 * Attributes:
 *     width (int): The width of the rectangle.
 *     height (int): The height of the rectangle.
 * 
 * Methods:
 *     area(): Calculate and return the area of the rectangle.
 *     perimeter(): Calculate and return the perimeter of the rectangle.
 */
public class Rectangle {
    private int width;
    private int height;

    /**
     * Constructs a Rectangle object with the specified width and height.
     * 
     * @param width The width of the rectangle.
     * @param height The height of the rectangle.
     * 
     * Example:
     *     Rectangle rect = new Rectangle(5, 10);
     *     rect.area(); // returns 50
     */
    public Rectangle(int width, int height) {
        this.width = width;
        this.height = height;
    }

    /**
     * Calculate and return the area of the rectangle.
     * 
     * @return The area of the rectangle.
     */
    public int area() {
        return width * height;
    }

    /**
     * Calculate and return the perimeter of the rectangle.
     * 
     * @return The perimeter of the rectangle.
     */
    public int perimeter() {
        return 2 * (width + height);
    }
}


Input: def greet_user(name, age):
    if age < 18:
        return f"Hello {name}, you are too young to register."
    else:
        return f"Welcome {name}, you are eligible to register."

Output:"""


## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


In [ ]:
print("Submitting generation request...")
generated_response = model.generate_text(prompt=prompt_input, guardrails=False)
print(generated_response)


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  